Importing section

In [67]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize

Reading the data from the folder and cleaning it.

In [68]:
sentences = []
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           s = s.lower()
           s = s.translate(str.maketrans('', '', string.punctuation))
           sentences.append(s)

tokensSentenceslist = []

for s  in sentences:
    wordsList = word_tokenize(s)
    tokensSentenceslist.append(wordsList)


##################### Uncomment below section for testing #########################
# print(len(sentences))
#
# for s in sentences:
#      print("The sentence is : ")
#      print(s)
#      print("-----------------------End of the sentence -------------")
#
# print (sentences)


# print (len(tokensSentenceslist))
# print (tokensSentenceslist)


Generating the Word2Vec Model

In [73]:
#model = Word2Vec(tokensSentenceslist, min_count=1)

model = Word2Vec(tokensSentenceslist, vector_size=50, min_count=1, sg=1)
#model = Word2Vec(sentences=tokensSentenceslist, vector_size=100, workers=1, seed=42)

##################### Uncomment below section for testing #########################


# print(list(model.wv.index_to_key))
# print(len(list(model.wv.index_to_key)))

In [66]:
model.wv.most_similar("argument")

[('aspect', 0.8933157324790955),
 ('point', 0.8480561971664429),
 ('reason', 0.8083563446998596),
 ('agrument', 0.7588935494422913),
 ('points', 0.7239507436752319),
 ('arguments', 0.7182546257972717),
 ('topic', 0.7170467972755432),
 ('second', 0.7071115970611572),
 ('contra', 0.6944943070411682),
 ('theme', 0.6861655712127686)]

In [72]:
model.wv.most_similar("argument")

[('aspect', 0.869485080242157),
 ('point', 0.8438014388084412),
 ('reason', 0.8218501210212708),
 ('pro', 0.743812084197998),
 ('statement', 0.7106698751449585),
 ('topic', 0.7102875113487244),
 ('arguments', 0.6799930334091187),
 ('contra', 0.6753125786781311),
 ('fact', 0.6655266284942627),
 ('second', 0.6605477929115295)]

In [74]:
model.wv.most_similar("argument")

[('point', 0.9051375985145569),
 ('aspect', 0.8784467577934265),
 ('reason', 0.8549717664718628),
 ('arguement', 0.8435595631599426),
 ('prohibition', 0.8274824023246765),
 ('contra', 0.8199025392532349),
 ('mention', 0.8168057799339294),
 ('speaks', 0.8067961931228638),
 ('pro', 0.7795742154121399),
 ('advantage', 0.7738142609596252)]